<a href="https://colab.research.google.com/github/AzlinRusnan/Optimizing-Customer-Satisfaction-CSAT-Through-Sentiment-Analysis-and-Predictive-ML-Techniques/blob/main/Optimizing_CSAT_Through_Sentiment_Analysis_%26_Predictive_ML_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

file_path = '/content/gdrive/MyDrive/Updated_CSAT_RAW_DATASET.xlsx'
xls = pd.ExcelFile(file_path)

# Check sheet names to understand the structure
xls.sheet_names

['Sheet1']

In [ ]:
# Load the data from the first sheet
df = pd.read_excel(xls, sheet_name='Page 1')

# Display the first few rows to understand the structure and locate the 'City' column
df.head()

,Number,Location,City,Country,Region,Updated,Average Response (calculated),USS Comment,String value
0,INC19296127,USPO,Pasco,United States of America,NORTH AMERICA,2024-10-31 23:18:02,1,\n\n\n\n,Very Satisfied
1,INC19297125,USGR,Greensboro,United States of America,NORTH AMERICA,2024-10-31 22:35:51,1,Thanks for punctual and quick service resolvin...,Very Satisfied
2,INC19283148,USGR,Greensboro,United States of America,NORTH AMERICA,2024-10-31 22:06:26,1,\n\n\n,Very Satisfied
3,INC19296794,BRSP,Sao Paulo,Brazil,LATAM,2024-10-31 21:39:18,1,\n\n\n,Very Satisfied
4,INC19295496,INPU,Pune,India,APAC,2024-10-31 21:26:21,1,\n\nPrompt response\n,Very Satisfied


In [ ]:
df.isnull().sum().to_frame().rename(columns={0:"Total No. of Missing Values"})

,Total No. of Missing Values
Number,0
Location,0
City,40380
Country,0
Region,0
Updated,0
Average Response (calculated),0
USS Comment,40536
String value,41040


In [ ]:
# To fill the missing 'City' values, I will first explore how to handle it.
# One approach is to use the most frequent city in the respective 'Location' or 'Country'.

# Check for the distribution of cities in the dataset
city_distribution = df['City'].value_counts()

# Checking the first few rows of 'Location' and 'City' to explore patterns
df[['Location','City']].head(10)

,Location,City
0,USPO,Pasco
1,USGR,Greensboro
2,USGR,Greensboro
3,BRSP,Sao Paulo
4,INPU,Pune
5,INPU,Pune
6,INPU,Pune
7,BRIF,São Paulo
8,MXMO,Los Mochis
9,GBGU,Guildford


In [ ]:
# I will attempt to fill missing 'City' values based on the most frequent city in each 'Location'.
city_mapping = df.groupby('Location')['City'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown').to_dict()

# Apply this mapping to fill missing 'City' values
df['City'] = df['City'].fillna(df['Location'].map(city_mapping))

# Check the number of missing values after filling
missing_values_after = df['City'].isnull().sum()
print("There are now " + str(missing_values_after) + " missing values in City!")

There are now 0 missing values in City!


In [ ]:
# Since we have fill in the gap in City. Lets proceed with missing values in String Value

# Check for missing values in the 'City' column
missing_values_sv = df['String value'].isnull().sum()
print("There are " + str(missing_values_sv)  + " missing values in the String Value column")

There are 41040 missing values in the String Value column


In [ ]:
# Let's check if there is any pattern to help us fill missing values in the 'String value' column
df[['String value']].drop_duplicates().head(20)

,String value
0,Very Satisfied
20,Satisfied
50,Very Dissatisfied
55,Neutral
70,Dissatisfied
49648,NaN


To fill the missing values in the "String value" column based on the "Average Response (calculated)" column, we can use the mapping:

1 → Very Satisfied

2 → Satisfied

3 → Neutral

4 → Dissatisfied

5 → Very Dissatisfied

In [ ]:
# Define the mapping
response_mapping = {
    1: "Very Satisfied",
    2: "Satisfied",
    3: "Neutral",
    4: "Dissatisfied",
    5: "Very Dissatisfied"
}

# Fill missing values in the "String value" column using the mapping
df['String value'] = df['String value'].fillna(
    df['Average Response (calculated)'].map(response_mapping)
)

# Verify the changes
df.tail()

,Number,Location,City,Country,Region,Updated,Average Response (calculated),USS Comment,String value
90683,INC6838845,INPU,Pune,India,APAC,2022-01-03 03:00:09,1,Thanks for fast action,Very Satisfied
90684,INC6390085,NLEN,Enkhuizen,Netherlands,EAME,2022-01-03 03:00:09,3,,Neutral
90685,INC6524346,USMN,Unknown,United States of America,NORTH AMERICA,2022-01-03 03:00:09,5,Issue not resolved.,Very Dissatisfied
90686,INC6968136,BRSP,Sao Paulo,Brazil,LATAM,2022-01-03 03:00:09,1,NaN,Very Satisfied
90687,INC7269810,IDKD,Kediri,Indonesia,APAC,2022-01-03 03:00:09,1,Fast Responses and helpful,Very Satisfied


In [ ]:
# Extract Year and Month-Year from the "Updated" column for us to have a better look on the output in the future

df['Year'] = pd.to_datetime(df['Updated']).dt.year
df['Month-Year'] = pd.to_datetime(df['Updated']).dt.strftime('%b/%Y')

df.tail()

,Number,Location,City,Country,Region,Updated,Average Response (calculated),USS Comment,String value,Year,Month-Year
90683,INC6838845,INPU,Pune,India,APAC,2022-01-03 03:00:09,1,Thanks for fast action,Very Satisfied,2022,Jan/2022
90684,INC6390085,NLEN,Enkhuizen,Netherlands,EAME,2022-01-03 03:00:09,3,,Neutral,2022,Jan/2022
90685,INC6524346,USMN,Unknown,United States of America,NORTH AMERICA,2022-01-03 03:00:09,5,Issue not resolved.,Very Dissatisfied,2022,Jan/2022
90686,INC6968136,BRSP,Sao Paulo,Brazil,LATAM,2022-01-03 03:00:09,1,NaN,Very Satisfied,2022,Jan/2022
90687,INC7269810,IDKD,Kediri,Indonesia,APAC,2022-01-03 03:00:09,1,Fast Responses and helpful,Very Satisfied,2022,Jan/2022


In [ ]:
# Remove the specified columns
columns_to_remove = ['Number', 'Location', 'Updated']
df = df.drop(columns=columns_to_remove)

df.head()

,City,Country,Region,Average Response (calculated),USS Comment,String value,Year,Month-Year
0,Pasco,United States of America,NORTH AMERICA,1,\n\n\n\n,Very Satisfied,2024,Oct/2024
1,Greensboro,United States of America,NORTH AMERICA,1,Thanks for punctual and quick service resolvin...,Very Satisfied,2024,Oct/2024
2,Greensboro,United States of America,NORTH AMERICA,1,\n\n\n,Very Satisfied,2024,Oct/2024
3,Sao Paulo,Brazil,LATAM,1,\n\n\n,Very Satisfied,2024,Oct/2024
4,Pune,India,APAC,1,\n\nPrompt response\n,Very Satisfied,2024,Oct/2024


In [ ]:
#df.to_excel('Updated_CSAT_RAW_DATASET.xlsx', index=False)

#from google.colab import files
#files.download('Updated_CSAT_RAW_DATASET.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **SENTIMENT ANALYSIS**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

file_path = '/content/gdrive/MyDrive/Sentiment Analysis_cleaned_dataset.xlsx'
xls = pd.ExcelFile(file_path)

# Check sheet names to understand the structure
xls.sheet_names

['Sheet1']

In [4]:
# Load the data from the first sheet
df = pd.read_excel(xls, sheet_name='Sheet1')

# Display the first few rows to understand the structure
df.head()

,City,Country,Region,Average Response (calculated),USS Comment,String value,Year,Month-Year,Language
0,Pasco,United States of America,NORTH AMERICA,1,NaN,Very Satisfied,2024,Oct/2024,NaN
1,Greensboro,United States of America,NORTH AMERICA,1,thanks punctual quick service resolving my issue,Very Satisfied,2024,Oct/2024,english
2,Greensboro,United States of America,NORTH AMERICA,1,NaN,Very Satisfied,2024,Oct/2024,NaN
3,Sao Paulo,Brazil,LATAM,1,NaN,Very Satisfied,2024,Oct/2024,NaN
4,Pune,India,APAC,1,prompt response,Very Satisfied,2024,Oct/2024,english


In [10]:
import pandas as pd

# Filter for English language and exclude rows with blank comments
filtered_df = df[(df['Language'] == 'english') & (df['USS Comment'].notna())]

filtered_df.head()

# Select relevant columns for BERT analysis
bert_data = filtered_df[['USS Comment', 'Average Response (calculated)']]

print(bert_data.head())

                                          USS Comment  \
1    thanks punctual quick service resolving my issue   
4                                     prompt response   
6                                       quick support   
10  andre provided excellent support he used teams...   
11  assistance was immediate resolved my issue alw...   

    Average Response (calculated)  
1                               1  
4                               1  
6                               1  
10                              1  
11                              1  


In [11]:
#pip install transformers

In [27]:
#from transformers import pipeline

# Initialize a sentiment-analysis pipeline using a pre-trained BERT model
#sentiment_analyzer = pipeline("sentiment-analysis")

# Perform sentiment analysis on the comments
#bert_data['Predicted Sentiment'] = bert_data['USS Comment'].apply(lambda x: sentiment_analyzer(x)[0]['label'])

# Adjust sentiment based on specific rules
#def adjust_sentiment(row):
#    comment = row['USS Comment'].lower()
#    sentiment = row['Predicted Sentiment']

    # Adjust misclassified positive comments
#    if "resolved" in comment and sentiment == "NEGATIVE":
#        return "POSITIVE"
#    return sentiment

#bert_data['Adjusted Sentiment'] = bert_data.apply(adjust_sentiment, axis=1)

# Map adjusted sentiment labels to numerical equivalents for comparison
#sentiment_mapping = {'POSITIVE': 1, 'NEGATIVE': 5, 'NEUTRAL': 3}
#bert_data['Sentiment Score'] = bert_data['Adjusted Sentiment'].map(sentiment_mapping)

# Detect mismatches between the sentiment score and the calculated response
#bert_data['Mismatch'] = bert_data['Sentiment Score'] != bert_data['Average Response (calculated)']

# Filter mismatched entries
#mismatched_data = bert_data[bert_data['Mismatch']]

# Count total mismatches
#mismatch_count = mismatched_data.shape[0]

# Display mismatched data and count
#print(f"Total Mismatches: {mismatch_count}")
#print(mismatched_data.head())


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Total Mismatches: 6165
                                          USS Comment  \
55  i expected faster solution as was asked treat ...   
70  my dissatisfaction because they didnt share ne...   
78  was not able resolve issue but issue resolved ...   
81  lisa reed was able resolve my issues with glob...   
92  resolution was sent but only after  hours not ...   

    Average Response (calculated) Predicted Sentiment  Sentiment Score  \
55                              3            NEGATIVE                5   
70                              4            NEGATIVE                5   
78                              3            NEGATIVE                1   
81                              1            NEGATIVE                5   
92                              3            NEGATIVE                5   

    Mismatch Adjusted Sentiment  
55      True           NEGATIVE  
70      True           NEGATIVE  
78      True           POSITIVE  
81      True           NEGATIVE  
92      True       